In [8]:
from pred_AAA import pred_AAA

In [1]:
from DataLoaders.Seg_DataLoader import Seg_DataLoader
from Generators.Seg_Generator import Seg_Generator
from Models.UNet3D import UNet3D_Extra
from Metrics.metrics import weighted_dice_coefficient_loss
from pred_AAA import pred_AAA
from config import config
import nibabel as nib
from DataUtils.Seg_tools import fast_process
import Metrics.eval_metrics as metrics
from Callbacks.LR_decay import get_callbacks
import numpy as np
from sklearn.model_selection import train_test_split
import keras

def compute_metrics(pred, truth):
    dc = metrics.dice_coef(pred, truth)
    iou = metrics.IOU(pred, truth)
     
    return [dc, iou]

def create_gens(train, test):
    
    gen = Seg_Generator(data_points = train,
                 dim=(1, 128, 128, 128),
                 batch_size = 1,
                 n_classes = 1,
                 shuffle = True,
                 augment = True,
                 distort = False,
                 permute = True
                 )

    test_gen = Seg_Generator(data_points = test,
                 dim=(1, 128, 128, 128),
                 batch_size = 1,
                 n_classes = 1,
                 shuffle = False,
                 augment = False)
   
    return gen, test_gen

def get_pre_items(names):

    pre_names = [name.replace('art', 'pre') for name in names]
    pre_names = [pre_name.replace('ven', 'pre') for pre_name in pre_names]

    dl_pre = Seg_DataLoader(init_location='/media/sage/data/nifti_endoleak/',
                            label_location=main_dr + 'labels/leak_segs/',
                            annotations=main_dr + 'labels/annotations.csv',
                            label_type='crop',
                            seg_key='garbage',
                            n_classes=1,
                            neg_list=pre_names,
                            in_memory=False,
                            memory_loc='/mnt/sda5/temp/',
                            preloaded=False)

    items = dl_pre.get_all()

    return items

print('Running')

TRAIN = False
EVAL = True
SAVE = True

add_extra = False

folds = 5
epochs = 200
threshold = .5

main_dr = '/home/sage/GenDiagFramework/'
loss_func = weighted_dice_coefficient_loss

dl = Seg_DataLoader(
        init_location = '/media/sage/data/nifti_endoleak/',
        label_location =  main_dr + 'labels/leak_segs/',
        annotations = main_dr + 'labels/annotations.csv',
        neg_list = None,
        in_memory = False,
        memory_loc = '/mnt/sda5/temp/',
        preloaded=False)

print('Loading training data+labels')
dl.setup_kfold_splits(folds, 43)

Using TensorFlow backend.


Running
Loading training data+labels


In [12]:
names = []

pre_leak_results = []
leak_results = []
post_leak_results = []
post_pre_leak_results = []
difs = []

vol_dict = {}
label_dict = {}

In [3]:
all_test = []

for fold in range(folds):
        
    model = UNet3D_Extra(input_shape = (1, 128, 128, 128), n_labels=1)
    model.compile(optimizer=keras.optimizers.adam(.001), loss=loss_func)

    train, test = dl.get_k_split(fold)

    extra = get_pre_items([t.get_name() for t in test])
    test += extra

    gen, test_gen = create_gens(train, test)
    model.load_weights(main_dr + 'saved_models/Leak-' + str(fold) + '.h5')

    print('*Predicting Leaks for fold - ', fold, '*')
    preds = model.predict_generator(test_gen)

    del model
    
    for p in range(len(preds)):
        test[p].set_pred_label(preds[p])
        all_test.append(test[p])

*Predicting Leaks for fold -  0 *
*Predicting Leaks for fold -  1 *
*Predicting Leaks for fold -  2 *
*Predicting Leaks for fold -  3 *
*Predicting Leaks for fold -  4 *


In [16]:
for dp in all_test:
    
    name = dp.get_name()
    pixdims = dp.get_pixdims()
    
    truth = np.squeeze(dp.get_label(copy=True))
    pred = dp.get_pred_label(True)
    
    pred[pred > threshold] = 1
    pred[pred < threshold] = 0

    pred = np.squeeze(pred)
    
    if 'pre' not in name:
        print(name)
        
        pre_leak_results.append(compute_metrics(pred, truth))
        print('pre_leak_results: ', pre_leak_results[-1])
        
        pred = fast_process(pred, .05)
        
        leak_results.append(compute_metrics(pred, truth))
        print('leak_results: ', leak_results[-1])
        
        print('*Predicting AAA*')
        AAA_dp = pred_AAA(name)
        AAA = AAA_dp.get_pred_label()
        
        G = np.zeros((128, 128, 128))
        G[AAA[0] == 1] = 1
        G[AAA[1] == 1] = 1

        AAA_intersect = np.sum(pred * G)
        preded_AAA = pred * G
        
        post_leak_results.append(compute_metrics(preded_AAA, truth))
        print('post_leak_results: ', post_leak_results[-1])

        AAA_intersect_volume = AAA_intersect * \
            (pixdims[0] * pixdims[1] * pixdims[2]) * 0.001 # .001 For mL
        print('AAA_intersect_volume: ', AAA_intersect_volume)

        True_volume = np.sum(truth) * (pixdims[0] * pixdims[1] * pixdims[2]) * 0.001
        print('True_volume: ', True_volume)
        
        dif = abs(AAA_intersect_volume - True_volume)
        difs.append(dif)

        vol_dict[name] = AAA_intersect_volume
        
    else:
        print(name)
        
        pred = fast_process(pred, .05)
        AAA_dp = pred_AAA(name)
        AAA = AAA_dp.get_pred_label()
        G = np.zeros((128, 128, 128))
        G[AAA[0] == 1] = 1
        G[AAA[1] == 1] = 1
        AAA_intersect = np.sum(pred * G)
        preded_AAA = pred * G
        
        AAA_intersect_volume = AAA_intersect * \
            (pixdims[0] * pixdims[1] * pixdims[2]) * 0.001 # .001 For mL
        print('pre_AAA_intersect_volume: ', AAA_intersect_volume)
        
        vol_dict[name] = AAA_intersect_volume
        

144b_ven
pre_leak_results:  [0.4546472620380739, 0.2942028985507246]
leak_results:  [0.4621514001138304, 0.3005181347150259]
*Predicting AAA*
post_leak_results:  [0.5918367419825072, 0.42028985507246375]
AAA_intersect_volume:  0.49346557617187503
True_volume:  0.4314453125
11c_ven
pre_leak_results:  [0.22498446861404597, 0.12675070028011204]
leak_results:  [0.23222440313111545, 0.13136531365313653]
*Predicting AAA*
post_leak_results:  [0.23222440313111545, 0.13136531365313653]
AAA_intersect_volume:  0.41019315719604493
True_volume:  0.6122883224487304
123a_ven
pre_leak_results:  [0.7099542063147747, 0.5503325610940887]
leak_results:  [0.7091683614096234, 0.5493887347183679]
*Predicting AAA*
post_leak_results:  [0.7091683614096234, 0.5493887347183679]
AAA_intersect_volume:  14.716306943893432
True_volume:  21.880475149154663
12a_ven
pre_leak_results:  [0.04964540189125296, 0.025454545454545455]
leak_results:  [0.05017922341696535, 0.025735294117647058]
*Predicting AAA*
post_leak_results

post_leak_results:  [0.6144781186868686, 0.44349939246658565]
AAA_intersect_volume:  0.7529641151428222
True_volume:  0.7317186546325684
153b_art
pre_leak_results:  [0.5285655579531442, 0.35921787709497205]
leak_results:  [0.5339422887689433, 0.36420277541772866]
*Predicting AAA*
post_leak_results:  [0.5339422887689433, 0.36420277541772866]
AAA_intersect_volume:  1.8525015115737915
True_volume:  4.95673143863678
15a_art
pre_leak_results:  [0.4484669412633912, 0.28904761904761905]
leak_results:  [0.4484669412633912, 0.28904761904761905]
*Predicting AAA*
post_leak_results:  [0.4484669412633912, 0.28904761904761905]
AAA_intersect_volume:  0.46308349609375
True_volume:  1.5204086303710938
15a_ven
pre_leak_results:  [0.04922516864175023, 0.025233644859813085]
leak_results:  [0.04922516864175023, 0.025233644859813085]
*Predicting AAA*
post_leak_results:  [0.04922516864175023, 0.025233644859813085]
AAA_intersect_volume:  0.02124908447265625
True_volume:  0.782552490234375
153a_ven
pre_leak_re

ResourceExhaustedError: OOM when allocating tensor with shape[1,16,128,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv3d_5763/convolution = Conv3D[T=DT_FLOAT, data_format="NCDHW", dilations=[1, 1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](up_sampling3d_1299/concat_2, conv3d_5763/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: activation_186/Sigmoid/_132795 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2748_activation_186/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, ad

In [17]:
print(np.mean(difs))
print(np.std(difs))

1.2013386963428214
1.944170561056283


In [18]:

print('pre Leak Means = ', np.mean(pre_leak_results, axis=0))
print('pre Leak stds = ', np.std(pre_leak_results, axis=0))
print('Leak Means = ', np.mean(leak_results, axis=0))
print('Leak stds = ', np.std(leak_results, axis=0))
print('post Leak Means = ', np.mean(post_leak_results, axis=0))
print('post Leak stds = ', np.std(post_leak_results, axis=0))

pre Leak Means =  [0.50908908 0.36762015]
pre Leak stds =  [0.21108204 0.18662337]
Leak Means =  [0.50958267 0.36934644]
Leak stds =  [0.21687204 0.19051983]
post Leak Means =  [0.52852241 0.3857989 ]
post Leak stds =  [0.21081061 0.18771664]


In [24]:
for name in vol_dict:
    if 'pre' not in name:
        try:

            pre_name = name.replace('art', 'pre')
            pre_name = pre_name.replace('ven', 'pre')

            print(name, ': ', end='')
            print(vol_dict[name] - vol_dict[pre_name])
        
        except:
            pass

144b_ven : -1.5855615234374998
11c_ven : 0.41019315719604493
123a_ven : 4.6335905413627625
12a_ven : -0.6387349196672439
144c_ven : 0.6100682258605956
11d_ven : 0.8646556288003922
13a_ven : -0.18276840233802805
14b_art : -0.5083653190135956
106b_art : 2.6251368970870974
144d_ven : -3.6461360996365544
142a_ven : 0.0548546531200409
128b_art : -1.720746804714203
106a_art : 1.6758282928466797
135a_ven : 4.210843551635742
135b_ven : 0.08061647415161133
128a_art : -0.3752664566040038
135c_ven : 2.4692046947479245
146a_ven : 2.7329156398773193
121c_art : 0.5658769773244857
14a_art : 0.18980271989107134
135d_ven : -0.059552764892578214
11c_art : 0.018008480072021486
135a_art : 2.558147157669067
166b_art : 3.0743408203125
163d_art : 179a_art : 153b_art : 15a_art : 15a_ven : 153a_ven : 168b_art : 166b_ven : 1.9064950942993164
174a_ven : 153b_ven : 183a_art : 174a_art : 149b_ven : 191a_ven : 163b_art : 168a_art : 153a_art : 180c_ven : 180a_ven : 177a_art : 180a_art : 

In [21]:
vol_dict

{'106a_art': 1.683189178466797,
 '106a_pre': 0.007360885620117188,
 '106b_art': 2.788205637931824,
 '106b_pre': 0.16306874084472656,
 '11c_art': 0.018008480072021486,
 '11c_pre': 0.0,
 '11c_ven': 0.41019315719604493,
 '11d_pre': 0.0,
 '11d_ven': 0.8646556288003922,
 '121c_art': 1.5135203665494918,
 '121c_pre': 0.9476433892250061,
 '123a_pre': 10.08271640253067,
 '123a_ven': 14.716306943893432,
 '128a_art': 1.8653180027008056,
 '128a_pre': 2.2405844593048094,
 '128b_art': 2.9236479949951173,
 '128b_pre': 4.64439479970932,
 '12a_pre': 1.2163574520349503,
 '12a_ven': 0.5776225323677063,
 '135a_art': 2.674562255859375,
 '135a_pre': 0.11641509819030763,
 '135a_ven': 4.3272586498260495,
 '135b_pre': 0.25838613510131836,
 '135b_ven': 0.3390026092529297,
 '135c_pre': 0.5593429412841797,
 '135c_ven': 3.0285476360321044,
 '135d_pre': 2.2407989501953125,
 '135d_ven': 2.1812461853027343,
 '13a_pre': 0.9166756303310395,
 '13a_ven': 0.7339072279930114,
 '142a_pre': 0.0,
 '142a_ven': 0.05485465312004